In [1]:
import os
for k, v in dict(
    EMAIL_TO='agalea91@gmail.com',
    EMAIL_FROM='Toc <toc.adi.now@gmail.com>',
    NOTION_DATABASE_ID='',
    NOTION_SECRET_TOKEN='',
    SENDGRID_SECRET_TOKEN=''
).items():
    os.environ[k] = v

## 2023-04

Grok the API

In [48]:
import requests
import os

HEADERS = {
    "Authorization": f"Bearer {os.getenv('NOTION_SECRET_TOKEN')}",
    "Notion-version": "2021-05-13",
}


def retrieve_database(database_id, query=None, start_cursor=None) -> dict:
    url = f"https://api.notion.com/v1/databases/{database_id}/query"
    data = query or {}
    if start_cursor:
        data["start_cursor"] = start_cursor
    print(f"POST {url}")
    response = requests.post(url=url, headers=HEADERS, json=data)
    response.raise_for_status()
    database = response.json()
    return database


def retrieve_page_content(page_id) -> dict:
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    print(f"GET {url}")
    response = requests.get(url=url, headers=HEADERS)
    response.raise_for_status()
    page = response.json()
    return page

In [49]:
import re
import random
import json


def get_random_page(database):
    if database["has_more"]:
        raise NotImplementedError("Database has more pages. Must implement pagination.")

    idx = random.choice(list(range(len(database["results"]))))
    page = database["results"][idx]
    return page


def get_page_id(database_page):
    return re.findall(r"/([^/]+)$", database_page["url"])[0]


def get_page_topic(database_page):
    return database_page["properties"]["Tags"]["multi_select"][0]["name"]


def parse_page_quote_content(block_page):
    try:
        quote = block_page["results"][0]["paragraph"]["text"][0]["text"]["content"]
        attrib = block_page["results"][1]["paragraph"]["text"][0]["text"]["content"]
    except Exception as e:
        print(f"Error parsing quote from page {json.dumps(block_page, indent=2)}")
        raise
    return quote, attrib

In [23]:

database1 = retrieve_database(
    database_id=os.getenv("NOTION_DATABASE_ID"),
    start_cursor=None
)

POST https://api.notion.com/v1/databases/bb7a2ecc460f43dbaf751cf1ee6d27ba/query


In [24]:
database1["has_more"]

True

In [43]:
database1["next_cursor"]

'59a3f846-0db7-48f3-89f9-d5ffd110922f'

In [50]:
database2 = retrieve_database(
    database_id=os.getenv("NOTION_DATABASE_ID"),
    start_cursor=database1["next_cursor"]
)

POST https://api.notion.com/v1/databases/bb7a2ecc460f43dbaf751cf1ee6d27ba/query


In [51]:
database2["has_more"]

False

In [52]:
len(database1["results"])

100

In [53]:

len(database2["results"])

6

In [54]:
database1["results"][0]["url"]

'https://www.notion.so/f04302363a184525b04e142cc3c1f1aa'

In [55]:

database2["results"][0]["url"]

'https://www.notion.so/59a3f8460db748f389f9d5ffd110922f'

In [58]:
type(database1)

dict

## 2023-07

Get full text of database page

In [22]:
import requests
import os
import time

MAX_PAGINATION = 100

HEADERS = {
    "Authorization": f"Bearer {os.getenv('NOTION_SECRET_TOKEN')}",
    "Notion-version": "2021-05-13",
}

def retrieve_database(database_id, query=None) -> dict:
    database = {"has_more": True, "next_cursor": None, "results": []}

    i = 0
    while database["has_more"]:
        i += 1
        if i > MAX_PAGINATION:
            raise NotImplementedError(f"Max pagination reached ({MAX_PAGINATION})")

        next_cursor = database.get("next_cursor")
        if not next_cursor and i > 1:
            break

        next_database = _retrieve_paginated_database(database_id, query, next_cursor)
        for k, v in next_database.items():
            if k == "results":
                database["results"].extend(v)
            else:
                database[k] = v

        time.sleep(1)
    return database


def _retrieve_paginated_database(database_id, query=None, start_cursor=None) -> dict:
    url = f"https://api.notion.com/v1/databases/{database_id}/query"
    data = query or {}
    if start_cursor:
        data["start_cursor"] = start_cursor
    print(f"POST {url} with data: {data}")
    response = requests.post(url=url, headers=HEADERS, json=data)
    response.raise_for_status()
    database = response.json()
    return database


def retrieve_page_content(page_id) -> dict:
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    print(f"GET {url}")
    response = requests.get(url=url, headers=HEADERS)
    response.raise_for_status()
    page = response.json()
    return page



def retrieve_page_content(page_id) -> dict:
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    print(f"GET {url}")
    response = requests.get(url=url, headers=HEADERS)
    if not response.ok:
        print(response.text)
        response.raise_for_status()
    page = response.json()
    return page

import re
import random
import json


def get_random_page(database):
    if database["has_more"]:
        raise NotImplementedError("Database has more pages. Must implement pagination.")

    idx = random.choice(list(range(len(database["results"]))))
    page = database["results"][idx]
    return page


def get_page_id(database_page):
    return re.findall(r"/([^/]+)$", database_page["url"])[0]


def get_page_topic(database_page):
    return database_page["properties"]["Tags"]["multi_select"][0]["name"]




In [12]:

database1 = retrieve_database(
    database_id=os.getenv("NOTION_DATABASE_ID"),
)

POST https://api.notion.com/v1/databases/6a58d3231dd24124b44ca8b42529795c/query with data: {}


In [14]:
database_page = get_random_page(database1)

In [50]:
print(json.dumps(database_page, indent=4))

{
    "object": "page",
    "id": "7be96769-67b4-48dc-a860-9bdf04816ea6",
    "created_time": "2023-07-02T01:24:00.000Z",
    "last_edited_time": "2023-07-02T01:42:00.000Z",
    "created_by": {
        "object": "user",
        "id": "8f75565e-980a-4508-92c6-154a3fa6afd2"
    },
    "last_edited_by": {
        "object": "user",
        "id": "8f75565e-980a-4508-92c6-154a3fa6afd2"
    },
    "cover": null,
    "icon": null,
    "parent": {
        "type": "database_id",
        "database_id": "6a58d323-1dd2-4124-b44c-a8b42529795c"
    },
    "archived": false,
    "properties": {
        "Origin": {
            "id": "GwJ]",
            "type": "multi_select",
            "multi_select": []
        },
        "Tags": {
            "id": "MQhk",
            "type": "multi_select",
            "multi_select": [
                {
                    "id": "a6c904d1-38ff-48ab-a929-9a2a10395b42",
                    "name": "SIBO",
                    "color": "green"
                }
     

In [51]:
database_page["properties"].get("Name", {})["title"][0]["text"]["content"]

'Energy Balls'

In [26]:
database_page["id"]

'7be96769-67b4-48dc-a860-9bdf04816ea6'

In [16]:
topic = get_page_topic(database_page)
topic

'SIBO'

In [35]:
block_page = retrieve_page_content(database_page["id"])

GET https://api.notion.com/v1/blocks/7be96769-67b4-48dc-a860-9bdf04816ea6/children


In [36]:
print(json.dumps(block_page, indent=4))

{
    "object": "list",
    "results": [
        {
            "object": "block",
            "id": "39438c20-36b3-4d8b-9aba-0b0a023f5ff5",
            "parent": {
                "type": "page_id",
                "page_id": "7be96769-67b4-48dc-a860-9bdf04816ea6"
            },
            "created_time": "2023-07-02T01:27:00.000Z",
            "last_edited_time": "2023-07-02T01:30:00.000Z",
            "created_by": {
                "object": "user",
                "id": "8f75565e-980a-4508-92c6-154a3fa6afd2"
            },
            "last_edited_by": {
                "object": "user",
                "id": "8f75565e-980a-4508-92c6-154a3fa6afd2"
            },
            "has_children": false,
            "archived": false,
            "type": "paragraph",
            "paragraph": {
                "color": "default",
                "text": [
                    {
                        "type": "text",
                        "text": {
                            "content": "

In [45]:
def parse_page_recipe_content(block_page):
    try:
        recipe = ""
        for r in block_page["results"]:
            result_type = r["type"]
            result_text_blocks = r[result_type]["text"]
            if not result_text_blocks:
                recipe += "\n"
            else:
                for t in result_text_blocks:
                    content = t['text']['content']
                    content_is_bold = t.get('annotations', {}).get("bold")
                    if result_type.startswith("heading"):
                        styled_content = f"<h2>{content}</h2>\n"
                    elif content_is_bold:
                        styled_content = f"<b>{content}</b>\n"
                    else:
                        styled_content = f"{content}\n"
                    recipe += styled_content
                        
                        
    except Exception as e:
#         print(f"Error parsing recipe from page {json.dumps(block_page, indent=2)}")
        raise
    return recipe





In [46]:
text = parse_page_recipe_content(block_page)
print(text)

<b>Base</b>
1/2 cup shredded coconut
1/2 cup walnuts
1/2 tsp ground cinnamon
1/4 tsp salt
2 Tbsp honey *
1/2 cup coconut oil
1/4 cup almond butter
1/4 cup plant milk

<h2>Flax & Pumpkin</h2>
1/2 cup flaxseed meal *
1/2 cup pumpkin seeds
2 tsp ground turmeric
1/4 tsp ground ginger

<b>Sunflower & Sesame</b>
1/2 cup sesame seeds
1/2 cup sunflower seeds
1/2 cup cocoa powder

* omit for Greystones reset

<b>Method</b>
Mix in blender
Let chill in fridge for 10 minutes
Store in airtight container for up to 1 month

